### Building blocks of RNN

Forward pass for Recurrent architecture. Using `torch.nn` module, a recurrent layer can be defined via **RNN**.

Create a recurrent layer from **RNN**, perform a forward pass on an input of lenght of 3 to compute the output.

Then, manually compute the forward pass and compare the results with those of RNN

#### Building an RNN model

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.rnn = nn.RNN(input_size, hidden_size, num_layers = 2, batch_first = True)
        # self.rnn = nn.GRU(input_size, hidden_size, num_layers = 2, batch_first = True)
        # self.rnn = nn.LSTM(input_size, hidden_size, num_layers = 2, batch_first = True)
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        _, hidden = self.rnn(x)
        out = hidden[-1, :, :] # we use the final hidden state from the last hidden layer as input to the fully connected layer
        out = self.fc(out)
        return out
    
model = RNN(64, 32)
print(model)
model(torch.randn(5, 3, 64))

#### More on Bidirectional RNN

In [ ]:
class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx = 0)
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first = True, bidirectional = True)
        self.fc1 = nn.Linear(rnn_hidden_size*2, fc_hidden_size)
        self_relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self_sigmoid = nn.Sigmoid()
    
    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy, enforce_sorted = False, batch_first = True)
        _, (hidden, cell) = self.rnn(out)
        out = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim = 1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        
        return out

In [ ]:
torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size)
model